In [ ]:
#imports
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("darkgrid")

In [ ]:
data = pd.read_excel('Memberships.xlsx', sheet_name = "Membership Transaction Data")

In [ ]:
column_keys = pd.read_excel('Memberships.xlsx')

In [ ]:
column_keys

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.isna().sum()/len(data)

In [ ]:
sns.heatmap(data.isna(), cbar = False)

In [ ]:
data['Members\Constituent\Address (Primary)\ZIP'].value_counts()

In [ ]:
#the columns need to be renamed to be more managable 
data.rename(columns={r"Members\Constituent\Lookup ID":"Memkey_ID",
                     "Membership ID":"Mem_ID",
       r"Members\Constituent\Address (Primary)\ZIP":"Zip",
       r"Membership Transaction\Membership\Membership program":"Program",
       r"Membership Transaction\Transaction date (date only)":"Transaction_date",
       r"Membership Transaction\Membership Level\Name":"Member_level",
       r"Membership Transaction\Expiration date (date only)":"Exp_date",
       r"Membership Transaction\Action":"Action", r"Membership Transaction\Is gift":"Gift",
       r"Membership Transaction\Membership\Status":"Status",
       r"Membership Transaction\Membership\Last renewed":"Last_renewed",
       r"Membership Transaction\Membership\Member since":"Mem_since"}, inplace = True)

In [ ]:
data.columns

In [ ]:
data.head(12)

In [ ]:
def categorical_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', verbose=True):
    '''
    Helper function that gives a quick summary of a given column of categorical data
    Arguments
    =========
    dataframe: pandas dataframe
    x: str. horizontal axis to plot the labels of categorical data, y would be the count
    y: str. vertical axis to plot the labels of categorical data, x would be the count
    hue: str. if you want to compare it another variable (usually the target variable)
    palette: array-like. Colour of the plot
    Returns
    =======
    Quick Stats of the data and also the count plot
    '''
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = dataframe[column_interested]
    print("***Descrition of Values***")
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print("***Total value counts of the series***")
        print(series.value_counts())
        print("***Ratio of values with respect to the total series***")
        print(series.value_counts()/(len(series)))

    sns.countplot(x=x, y=y, hue=hue, data=dataframe, palette="RdBu_r")
    plt.show()


In [ ]:
categorical_summarized(data, y = 'Member_level')

In [ ]:
data.columns

In [ ]:
import datetime as dt

In [ ]:
data.Exp_date = pd.to_datetime(data.Exp_date)

In [ ]:
data.Transaction_date.head(7)

In [ ]:
#I am now going to subset the data to pull out the members who are mission drive, and not as much value driven, 
#taking out the family and grandparents, and leaving the rest. 

In [ ]:
progressing_mems = data[(data.Member_level != 'Family') & (data.Member_level != 'Grandparents')]

In [ ]:
#We're now going to do the same catagorical analysis, but with the subsetted data

In [ ]:
categorical_summarized(progressing_mems, y = 'Member_level')

In [ ]:
#who is this 1 life membership?
Life = data[data.Member_level == 'Life']

In [ ]:
Life #this does't really say much

In [ ]:
categorical_summarized(data, y = 'Status')

In [ ]:
categorical_summarized(data, y = 'Program')

In [ ]:
#now I am going to pull in a new data set which contains member tansactions ie nat talk, GA admissions, special exhibit

In [ ]:
memsales = pd.read_excel('Sales.xlsx', sheet_name = "Member Sales Data")

In [ ]:
memsales_key = pd.read_excel('Sales.xlsx')

In [ ]:
memsales_key

In [ ]:
memsales.head()

In [ ]:
memsales.columns

In [ ]:
memsales.rename(columns={r"Constituent\Member\Constituent\Lookup ID":"Memkey_ID",
                     r"Constituent\Member\Membership\Membership ID":"Mem_ID",
                    r'Sales Order Item\Sales Order Item Ticket\Program\Name':"Program_name",
       r'Sales Order Item\Sales Order Item Ticket\Program Events\Name':"Event_name",
       r'Sales Order Item\Sales Order Item Ticket\Price type code':"Ticket_type",
       r'Sales Order Item\Quantity':"Quantity", r'Transaction date':"Date"
       }, inplace = True)

In [ ]:
memsales.head()

In [ ]:
#data quality check
memsales.isna().sum()/len(memsales)

In [ ]:
combined_data = data.merge(memsales, how = 'outer', on = 'Memkey_ID')

In [ ]:
combined_data.columns

In [ ]:
#let's take a look and see missing fields as members who did not engage in any program in the data, yet remained
# a member

In [ ]:
sns.heatmap(combined_data.isna(), cbar = False)

In [ ]:
combined_data.isna().sum()/len(combined_data)

In [ ]:
#roughly 30% are not actively participating in the museum. 

In [ ]:
categorical_summarized(combined_data, y = 'Program_name')

In [ ]:
NonGA = combined_data[combined_data.Program_name != "General Admission"]


In [ ]:
GA = combined_data[(combined_data.Program_name == "General Admission") 
                   | (combined_data.Program_name == "Ultimate Dinosaurs")]

In [ ]:
GA.head(9)

In [ ]:
MemID_GA = GA.Memkey_ID.unique()

In [ ]:
type(MemID_GA)

In [ ]:
MemID_GA.shape

In [ ]:
combined_data.shape

In [ ]:
# This method does not seem to work due to dimmensionality of the MemID_CA
#MissionFocused = combined_data[combined_data.Memkey_ID != MemID_GA]

In [ ]:
MissionFocused = combined_data[~combined_data.Memkey_ID.isin(MemID_GA)]

In [ ]:
MissionFocused.shape

In [ ]:
NonGA.isna().sum()/len(NonGA)

In [ ]:
#let's assume now that a lot of the NaN are a result of non-active memberships. Finding NaN in the program field
#of ACTIVE memberships could be a lot more useful and insightful

In [ ]:
#let's first do a qualitative analysis of the Status column

In [ ]:
categorical_summarized(combined_data, y = 'Status')

In [ ]:
combinedactive = combined_data[combined_data.Status == "Active"]

In [ ]:
#Now we only have active memberships, let's see the member behavior of these members, and if any are non-participating

In [ ]:
categorical_summarized(combinedactive, y = 'Program_name')

In [ ]:
combinedactive.isna().sum()

In [ ]:
combinedactive.isna().sum()/len(combinedactive)

In [ ]:
#twelve percent of active memberships have not engaged in a program

In [ ]:
#let's now take out GA ( and ultimate Dinos)

In [ ]:
ActNonGA = combinedactive[(combinedactive.Program != "General Admission") 
                          & (combinedactive.Program != "Ultimate Dinosaurs")]

In [ ]:
#let's now repeate the process of this more refined group who are more mission driven. 

In [ ]:
ActNonGA.isna().sum()/len(ActNonGA)

In [ ]:
categorical_summarized(MissionFocused, y = 'Program_name')

In [ ]:
#Trying to isolate the active but nonparticiparint members

In [ ]:
#combinedactive.Program_name

In [ ]:
nonparticipating = combinedactive[combinedactive.Program_name.isna()]

In [ ]:
sns.heatmap(nonparticipating.isna(), cbar = False)

In [ ]:
nonparticipating.shape

In [ ]:
#lets take a look at the active participating and merge the membeships onto it. 

In [ ]:
combinedactive.Memkey_ID

In [ ]:
# I was told I could merge on this column with the donations, but we'll see if that's possible


In [ ]:
#read in donation data
donationdata = pd.read_excel('DonationsSDNHM.xlsx', sheet_name = "Donation Information")

In [ ]:
#combinedactive.merge(donationdata, left_on = 'Memkey_ID', right_on ='Recognition Lookup ID', suffixes='_left')

#so this didn't work. I need to dondence down the donation data

In [ ]:
donationdata.head()

In [ ]:
groupeddon = donationdata.groupby(['Recognition Lookup ID', 'Recognition Name'])['Recognition Amount'].sum()

In [ ]:
aggdondata= pd.DataFrame(groupeddon)

In [ ]:
aggdondata.head()

In [ ]:
aggdondata.reset_index(inplace=True)

In [ ]:
#lets re-name the primary key to the other datasets
aggdondata.rename(columns={'Recognition Lookup ID':'Memkey_ID'},inplace= True )

In [ ]:
aggdondata.head()

In [ ]:
#lets try the merge again, but with slightly different syntax
combined_donmemdata = combinedactive.merge(aggdondata, how = 'left', on = 'Memkey_ID')

In [ ]:
combined_donmemdata.head(12)

In [ ]:
combined_donmemdata['Recognition Amount'] = combined_donmemdata['Recognition Amount'].fillna(0)

In [ ]:
combined_donmemdata.head(8)

In [ ]:
sns.heatmap(combined_donmemdata.isna(), cbar = False)

In [ ]:
#ok lets create a new column adding a qualitative 'yes' or 'no' if they donated

In [ ]:
combined_donmemdata.columns

In [ ]:
def donation_check(x):
    if x != 0:
        return 'Yes'
    else:
        return 'No'
    
#apply to dataframe-
combined_donmemdata['Member_Donations'] = combined_donmemdata['Recognition Amount'].apply(donation_check)

In [ ]:
combined_donmemdata.head()

In [ ]:
#lets now see which members are participating in the museum and add another column to indicate if they are passive

In [ ]:
#let's take a look at missing values again. The NaN of programe_name are those who are not participating. 

In [ ]:
combined_donmemdata.isna().sum()

In [ ]:
categorical_summarized(combined_donmemdata, y = 'Status') #so this indicates they are indeed all "active" members

In [ ]:
def participatoin_check(x):
    if x == 'NaN':
        return 'No'
    else:
        return 'Yes'
    
#apply to dataframe-
combined_donmemdata['Member_Participation'] = combined_donmemdata['Program_name'].apply(participatoin_check)

In [ ]:
combined_donmemdata.head(9)

In [ ]:
def missionmem_check(x):
    """creating a new variable of members who are not family/grandparents. This helps indicate mission driven mems"""
    if x == 'Family':  
        return 'No'
    elif x == 'Grandparents':
        return 'No'
    else:
        return 'Yes'
    
#apply to dataframe-
combined_donmemdata['Mission_focused_mems'] = combined_donmemdata['Member_level'].apply(missionmem_check)

In [ ]:
combined_donmemdata.columns

In [ ]:
#combined_donmemdata.head(12)

In [ ]:
#just making sure the family and grandparents did not drop out and are still part of the data
#categorical_summarized(combined_donmemdata, y = 'Member_level')

In [ ]:
combined_donmemdata.columns

In [ ]:
MemID = (MemID_GA).tolist()

In [ ]:
type(MemID)

In [ ]:
def nonga_check(x):
   
    """creating a new variable of members who do not go to GA. Indicates more mission or BRCC focused members regardless of membership level"""
   
    if x == MemID:  #this were the key_ids of members who checked in under GA. this was used previously
        return 'No'
    else:
        return 'Yes'
    
#apply to dataframe-
combined_donmemdata['NonGA'] = combined_donmemdata['Memkey_ID'].apply(nonga_check)

In [ ]:
combined_donmemdata['NonGA'] = ~combined_donmemdata['Memkey_ID'].isin(MemID_GA)

In [ ]:
combined_donmemdata.columns

In [ ]:
#creating a new DF that isolates mems who also donate
memsanddons = combined_donmemdata[combined_donmemdata['Recognition Amount'] >0]

In [ ]:
memsanddons['Recognition Amount'].describe()

In [ ]:
from pandas import ExcelWriter

In [ ]:
#exporting the data to excel Pulling in various dataframes from the project that segmented the members differetly. 
#with ExcelWriter('Member_Segments.xlsx') as writer:
    #combined_donmemdata.to_excel(writer, sheet_name='All_Active')
    #combined_data.to_excel(writer, sheet_name='Combined_Member_Sales')
    #MissionFocused.to_excel(writer, sheet_name='Non_GA')
    #nonparticipating.to_excel(writer, sheet_name='Nonparticipant')
    #memsanddons.to_excel(writer, sheet_name = 'Mems_Who_Donate')

In [ ]:
#lets now isolate the zip codes of grandparents and failies for potential, future marketing 

In [ ]:
zip_codes = data[(data.Member_level == 'Family') |
                 (data.Member_level == 'Grandparents')]

"""This also includes lapsed memberships"""

In [ ]:
zip_codes.columns

In [ ]:
zip_codes.head()

In [ ]:
#making sure I just have family and grandparents memberships, and no other types
categorical_summarized(zip_codes, y = 'Member_level')

In [ ]:
#isolate the relevant variables
zip_codes = zip_codes[['Memkey_ID','Member_level', 'Zip','Status']]

In [ ]:
zip_codes.head()

In [ ]:
zip_totals = pd.DataFrame(zip_codes.Zip.value_counts())


In [ ]:
sorted(zip_totals)

In [ ]:
zip_totals = zip_totals.reset_index()

In [ ]:
zip_totals = zip_totals.rename(columns={'index':'zip','Zip':'mem_count'})

In [ ]:
zip_totals.head()

In [ ]:
#export zip codes of family/grandparents sperately. 
#with ExcelWriter('Fam_GP_Zip.xlsx') as writer:
    #zip_codes.to_excel(writer, sheet_name ='Zip_Codes')
    #zip_totals.to_excel(writer, sheet_name = 'Zip_counts')

In [ ]:
#export zip codes of family/grandparents sperately. 
#with ExcelWriter('Fam_GP_Zip.xlsx') as writer:
    #zip_codes.to_excel(writer, sheet_name ='Zip_Codes')
    #zip_totals.to_excel(writer, sheet_name = 'Zip_counts')

In [ ]:
combined_donmemdata.columns

In [ ]:

categorical_summarized(combined_donmemdata, y = 'NonGA')

In [ ]:
combined_donmemdata.columns

In [ ]:
new = combined_donmemdata[~ combined_donmemdata.Memkey_ID.isin(MemID_GA)]

In [ ]:
type(MemID_GA)

In [ ]:
MemID_GA.shape

In [ ]:
memsanddons.head()

In [ ]:
categorical_summarized(memsanddons, y = 'Member_level')

In [ ]:
memsanddons.columns

In [ ]:
groupedmemsdons = memsanddons.groupby(['Memkey_ID','Recognition Name','Member_level'])['Recognition Amount'].agg([np.sum, np.mean, np.std, np.ma.count])
#turns out this is not correct. The Recognition Amount is already a sum, which is why STV is zero. 

In [ ]:
groupedmemsdons.head(30)

In [ ]:
topgroupedmemdons = groupedmemsdons = memsanddons.groupby(['Memkey_ID','Recognition Name','Member_level'])['Recognition Amount'].agg([np.sum, np.mean, np.std])

In [ ]:
topgroupedmemdons.head()

In [ ]:
#Fascinating. the STD is zero for all becasue it was already aggregated when appending it to the memberships

In [ ]:
#Let's try this again, collapsing the data set but not createing eroneous aggregations 

In [ ]:
memsanddons.head()

In [ ]:
bettergroup = memsanddons.groupby(['Memkey_ID','Recognition Name','Member_level'])['Recognition Amount'].agg([np.mean])

In [ ]:
bettergroup = pd.DataFrame(bettergroup)

In [ ]:
bettergroup = bettergroup.rename(columns={'mean':'TotalMemberDonations'})

In [ ]:
betterdonemem = bettergroup.sort_values(by = 'TotalMemberDonations', ascending = False)

In [ ]:
betterdonemem

In [ ]:
MissionFocused.head()

In [ ]:
categorical_summarized(MissionFocused, y = 'Program_name')

In [ ]:
missiongrouped = MissionFocused.groupby('Memkey_ID').Member_level.unique()

#These are people who do not show up to general admission

In [ ]:
missiongrouped = pd.DataFrame(missiongrouped)

In [ ]:
missiongrouped.head(10);

In [ ]:
nonparticipating.head(16);

In [ ]:
#take a look and aggregate down thsoe who rejoin or join but have no activity in the museum

In [ ]:
npgrouped = nonparticipating.groupby('Memkey_ID').Member_level.unique()

In [ ]:
npgrouped = pd.DataFrame(npgrouped)

In [ ]:
categorical_summarized(progressing_mems, y = 'Member_level')

In [ ]:
progressgrouped = progressing_mems.groupby('Memkey_ID').Member_level.unique()

In [ ]:
progressgrouped = pd.DataFrame(progressgrouped)

In [ ]:
progressgrouped

In [ ]:
#exporting the data to excel Pulling in various dataframes from the project that segmented the members differetly. 
#with ExcelWriter('member_segments_2020_03_11.xlsx') as writer:
    #combined_donmemdata.to_excel(writer, sheet_name='All_Active')
    #betterdonemem.to_excel(writer, sheet_name='members_who_donate')
    #missiongrouped.to_excel(writer, sheet_name='Non_GA_Participants')
    #npgrouped.to_excel(writer, sheet_name='nonparticipants')
    #progressgrouped.to_excel(writer, sheet_name = 'Non_Fam_GP')

In [ ]:
combined_donmemdata.columns

In [ ]:
# lets create a new data set with unique member values, but with everyone with an active membership
CombinedDonmemDataActive = combined_donmemdata[combined_donmemdata.Status == 'Active']

In [ ]:
GroupedActiveComb = pd.DataFrame(CombinedDonmemDataActive.groupby(['Memkey_ID','Recognition Name',
                                                                   'Member_level', 'Status', 'Exp_date', 'Zip'])['Recognition Amount'].agg([np.sum]))

In [ ]:
GroupedActiveComb

In [ ]:
combined_donmemdata.columns

In [ ]:
#creating a further subset and differentiation for those who are current 
CurrentCombined = combined_donmemdata[combined_donmemdata.Exp_date > '2020-03-31']

In [ ]:
GroupedCurrent = pd.DataFrame(CurrentCombined.groupby(['Memkey_ID','Recognition Name',
                                                                   'Member_level', 'Status', 'Exp_date', 'Zip'])['Recognition Amount'].agg([np.sum]))

In [ ]:
GroupedCurrent = GroupedCurrent.reset_index()

In [ ]:
#renaming some columns

GroupedCurrent = GroupedCurrent.rename(columns = { 'Zip':'Zip Code', 'sum':'Total Donations'})

In [ ]:
GroupedCurrent

In [ ]:
#This current group has donations, but they are all positive. Does not include members who did not donate

In [ ]:
#SimpleCurrentGrouped = pd.DataFrame(CurrentCombined.groupby(['Memkey_ID','Member_level','Zip','Recognition Name']))

In [ ]:
#SimpleCurrentGrouped

In [ ]:
CurrentCombined.columns

In [ ]:
CurrentCombined.sort_values('Memkey_ID', inplace = True)

In [ ]:
# The reason why it was only pulling members is because I had "Recognition Name" included which are only members
GroupedCurrentAll = pd.DataFrame(CurrentCombined.groupby(['Memkey_ID','Member_level',
                                                       'Status', 'Exp_date', 'Zip'])['Recognition Amount'].agg([np.sum]))

In [ ]:
GroupedCurrentAll.reset_index(inplace = True)

In [ ]:
GroupedCurrentAll = GroupedCurrentAll.rename(columns= {'Zip':'Zip Code','sum':'Donation Total of Member'})

In [ ]:
GroupedCurrentAll

In [ ]:
#Awesome! I'll use this new data for the google maps!
#GroupedCurrentAll.to_excel('NewZipCodes.xlsx')

In [ ]:
GroupedCurrentAll = GroupedCurrentAll.sort_values('Donation Total of Member', ascending = False)

In [ ]:
GroupedCurrentAll.to_excel('CurrentActiveMembers.xlsx')